Football Stats Profile Notebook
===
Independent Variables (Features) = Variables that we are going to use to predict (ex.: possession, shots, etc)

Dependent Variable = Variables that we want to predict (final score)

In [ ]:
import pandas

In [ ]:
STATS_FILE = "resources/Chelsea-Stats.csv"

In [ ]:
df = pandas.read_csv(STATS_FILE, sep=";")
print(df.head())